## Aufgabe 1

### 8-Queens Problem

Das Spielfeld wird in einem Vektor mit 8 Dimensionen dargestellt, wobei gilt:

#### Kodierung

$$
Z \in \mathbb{N} := \{x \in \mathbb{N} \mid 0 <= x <= 7\}
$$

$$
V = (Z_1, Z_2, Z_3, Z_4, Z_5, Z_6, Z_7, Z_8)
$$

Der Index des Vektors ist die X-Koordinate und der Wert ist die Y-Koordinate des Spielfelds. Dadurch lässt sich das Spielfeld abbilden.

#### Operatoren

Crossover: Es werden zwei Individuen ausgewählt, basierend auf ihrer Fitnessfunktion, und an einer zufälligen Stelle gespalten und zusammengeführt. Für die Auswahl der Individuen wird die Glücksrad methode verwendet. Bessere individuen werden häufiger in dem Rad vertreten durch diese funktion $f(x)=(8-x)^2$ wobei x der fittniswert des individums ist.

Mutation: Nach dem Crossover hat jedes neue Individuum die Chance, einen zufälligen Wert durch einen anderen zufälligen Wert zu ersetzen.

#### Fitness-Funktion

Bei der Fitness-Funktion prüft jede Dame, ob sie geschlagen werden kann. Die Anzahl der Damen, die geschlagen werden können, ist der Wert der Funktion.

Das bedeutet: Je geringer der Wert, desto besser das Individuum.

#### Simulated Annealing

???

### Karte färben

#### Kodierung

Für die Länder die aneinander gränzen wird eine statisches dictionary benutzt, das als key den index des landes hat und als value ein Array mit den index der Länder die angränzen.

die Individuen werden in einem Array mit der länder der länder gespeichert.Jedem Land wird eine index im Array zugewiesen.
In dem Array wird die Farbe gespeichert mit der das Land eingefärbt ist.

die Farben werden in einem Seperatem Array gespeichert.
#### Operatoren
Crossover: Es werden zwei Individuen ausgewählt, basierend auf ihrer Fitnessfunktion, und an einer zufälligen Stelle gespalten und zusammengeführt. 

Mutation: Nach dem Crossover hat jedes neue Individuum die Chance, die Farbe eines zufälligen landes,  durch eine andere zufälligen Farbe zu ersetzen.

#### Fitniss-Funktion
Für die Fitniss werden die länder gezählt die ein nachbarland mit der selben Farbe haben.

## Aufgabe 2
### 8-Queens Problem

In [20]:
import random
random.seed()

class Board:
    def __init__(self, positions):
        self.positions = positions

    def fitniss(self):

        marked = [[False]*8 for _ in range(8)]
        for i in range(8):
            x = i
            y = self.positions[i]
            for j in range(8):
                if (j != y):
                    marked[x][j] = True
                if (j != x):
                    marked[j][y] = True
            
            temp_x = x
            temp_y = y
            for _ in range(8):
                temp_x = temp_x - 1
                temp_y = temp_y - 1
                if (temp_x < 0 or temp_y < 0):
                    break
                marked[temp_x][temp_y] = True

            temp_x = x
            temp_y = y
            for _ in range(8):
                temp_x = temp_x + 1
                temp_y = temp_y - 1
                if (temp_x > 7 or temp_y < 0):
                    break
                marked[temp_x][temp_y] = True

            temp_x = x
            temp_y = y
            for _ in range(8):
                temp_x = temp_x - 1
                temp_y = temp_y + 1
                if (temp_x < 0 or temp_y > 7):
                    break
                marked[temp_x][temp_y] = True
            
            temp_x = x
            temp_y = y
            for _ in range(8):
                temp_x = temp_x + 1
                temp_y = temp_y + 1
                if (temp_x > 7 or temp_y > 7):
                    break
                marked[temp_x][temp_y] = True
                
        counter = 0
        for i in range(8):
            x = i
            y = self.positions[i]
            if (marked[x][y]):
                counter = counter + 1
        return counter

    def mutate(self):
        mutate_chance = random.randint(1,1000)

        if (mutate_chance <= 50):
            mutate_index = random.randint(0,7)
            self.positions[mutate_index] = random.randint(0,7)

    def printBoard(self):
        pos = [[False]*8 for _ in range(8)]
        for i in range(8):
            x = i
            y = self.positions[i]
            pos[x][y] = True

        for row in pos:
            print(" ".join('x' if cell else '_' for cell in row))
        print()

        


def crossover(board1, board2):
    point = random.randint(1,6)
    child1 = board1.positions[:point] + board2.positions[point:]
    child2 = board2.positions[:point] + board1.positions[point:]
    return Board(child1), Board(child2)

def select(boards, numBoardsTotal):
    new_boards: Board = []
    selection_pool = []
    for i in range(len(boards)-1):
        selection_value = 8-boards[i].fitniss()
        for _ in range(selection_value ** 2):
            selection_pool.append(i)

    for _ in range(numBoardsTotal // 2):
        index1 = random.randint(0,len(selection_pool)-1)
        index2 = random.randint(0,len(selection_pool)-1)

        while(selection_pool[index1] == selection_pool[index2]):
            index2 = random.randint(0,len(selection_pool)-1)

        boardindex1 = selection_pool[index1]
        boardindex2 = selection_pool[index2]

        board1, board2 = crossover(boards[boardindex1], boards[boardindex2])

        board1.mutate()
        board2.mutate()
        new_boards.append(board1)
        new_boards.append(board2)

    return new_boards

def Evo(boards, generations, numBoardsTotal):
    for g in range(generations):
        boards = sorted(boards, key= lambda x: x.fitniss())
        print(boards[1].fitniss())

        new_boards: Board = select(boards, numBoardsTotal)


        boards = new_boards

        for b in boards:
            if b.fitniss() == 0:
                return b
    
    boards = sorted(boards, key= lambda x: x.fitniss())
    return boards[1]



boards: Board = []
numBoardsTotal = 100
for i in range(numBoardsTotal):
    boards.append(Board([random.randint(0,7) for _ in range(8)]))

generations = 20


b : Board = Evo(boards, generations, numBoardsTotal)

b.printBoard()
b.fitniss()


        


5
4
3
2
_ _ _ x _ _ _ _
_ x _ _ _ _ _ _
_ _ _ _ _ _ _ x
_ _ _ _ _ x _ _
x _ _ _ _ _ _ _
_ _ x _ _ _ _ _
_ _ _ _ x _ _ _
_ _ _ _ _ _ x _



0

## Aufgabe 2
Karte Färben

In [274]:
import random

random.seed()

nachbarn = {
    0:[1,2],
    1:[0,2,3],
    2:[0,1,3],
    3:[1,2,4],
    4:[3],
    5:[],
}

color = ["White", "Black", "Red", "Blue", "Green"]

class Karte:
    def __init__(self, länder):
        self.länder = länder
        pass

    def fitness(self):
        counter = 0
        colors_used = []
        for i in range(len(self.länder)):
            current_col = self.länder[i]
            if current_col not in colors_used:
                colors_used.append(current_col)
            for j in nachbarn[i]:
                if (current_col == self.länder[j]):
                    counter += 1
        counter += len(colors_used)
        
        return counter
    
    def mutate(self):
        mutate_chance = random.randint(1,1000)

        if (mutate_chance <= 50):
            mutate_index = random.randint(0,4)
            mutate_index = random.randint(0,4)
            c = color[random.randint(0,4)]
            self.länder[mutate_index] = c

    def printColors(self):
        print(self.länder)
    

def crossover(board1, board2):
    point = random.randint(1,6)
    child1 = board1.länder[:point] + board2.länder[point:]
    child2 = board2.länder[:point] + board1.länder[point:]
    return Karte(child1), Karte(child2)

def select(karten, numBoardsTotal):
    new_karte: Karte = []
    selection_pool = []
    for i in range(len(karten)-1):
        selection_value = 10-karten[i].fitness()
        for _ in range(selection_value ** 2):
            selection_pool.append(i)

    for _ in range(numBoardsTotal // 2):
        index1 = random.randint(0,len(selection_pool)-1)
        index2 = random.randint(0,len(selection_pool)-1)

        while(selection_pool[index1] == selection_pool[index2]):
            index2 = random.randint(0,len(selection_pool)-1)

        kartenindex1 = selection_pool[index1]
        kartenindex2 = selection_pool[index2]

        karte1, karte2 = crossover(karten[kartenindex1], karten[kartenindex2])

        karte1.mutate()
        karte2.mutate()
        new_karte.append(karte1)
        new_karte.append(karte2)

    return new_karte
    
def Evo(karten, generations, numKartenTotal):
    for g in range(generations):
        karten = sorted(karten, key= lambda x: x.fitness())
        #print(karten[1].fitness())

        new_karten: Board = select(karten, numKartenTotal)

        karten = new_karten

        for k in karten:
            if k.fitness() <= 3:
                print("found in generation:" + str(g))
                return k
    
    karten = sorted(karten, key= lambda x: x.fitness())
    print("not found")
    return karten[1]

karten: Karte = []

numBoardsTotal = 4
generations = 1000

for i in range(numBoardsTotal):
    karten.append(Karte([color[random.randint(0,4)] for _ in range(5)]))

karte = Evo(karten,generations,numBoardsTotal)

print("=============")
karte.printColors()
print(karte.fitness())

found in generation:119
['Red', 'Black', 'Blue', 'Red', 'Black']
3
